<a href="https://colab.research.google.com/github/mansfire/model_1_signal/blob/main/examplemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import pyplot
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from collections import Counter
from keras import callbacks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout,InputLayer
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend
from random import shuffle
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#Read in the data from excel (I converted form csv)
data = pd.read_excel('/content/drive/MyDrive/New_data.xlsx')

In [4]:
data.head(5)

,Unnamed: 0,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,...,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start,path_loss,signal_after
0,0,2.4,0,0,0,-5.0,-5.0,-5.0,8.660254,180,...,0.000000,0,0,1,3.209568,2,1,246,46.939065,199.060935
1,1,5.0,0,0,0,-5.0,-5.0,-4.5,8.381527,180,...,0.000000,0,0,1,0.924371,3,1,242,52.182334,189.817666
2,2,2.4,0,0,0,-5.0,-5.0,-4.0,8.124038,180,...,4.412869,2,1,0,0.000000,0,0,244,43.800806,200.199194
3,3,5.0,0,0,0,-5.0,-5.0,-3.5,7.889867,90,...,0.000000,0,0,0,0.000000,0,0,242,51.690673,190.309327
4,4,2.4,0,0,0,-5.0,-5.0,-3.0,7.681146,0,...,1.955627,3,3,0,0.000000,0,0,253,51.233753,201.766247


In [5]:
data=data.drop(['Unnamed: 0'],axis=1)

In [6]:
data.head(5)

,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,...,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start,path_loss,signal_after
0,2.4,0,0,0,-5.0,-5.0,-5.0,8.660254,180,0,...,0.000000,0,0,1,3.209568,2,1,246,46.939065,199.060935
1,5.0,0,0,0,-5.0,-5.0,-4.5,8.381527,180,270,...,0.000000,0,0,1,0.924371,3,1,242,52.182334,189.817666
2,2.4,0,0,0,-5.0,-5.0,-4.0,8.124038,180,270,...,4.412869,2,1,0,0.000000,0,0,244,43.800806,200.199194
3,5.0,0,0,0,-5.0,-5.0,-3.5,7.889867,90,90,...,0.000000,0,0,0,0.000000,0,0,242,51.690673,190.309327
4,2.4,0,0,0,-5.0,-5.0,-3.0,7.681146,0,0,...,1.955627,3,3,0,0.000000,0,0,253,51.233753,201.766247


In [7]:
#finding the null values
data.isnull().sum()

Frequency              0
router_x               0
router_y               0
router_z               0
cell_x                 0
cell_y                 0
cell_z                 0
Distance               0
orientation_x          0
orientation_y          0
orientation_z          0
orientation_overall    0
pass_0                 0
pass_dis_0             0
pass_type_0            0
pass_mat_0             0
pass_1                 0
pass_dis_1             0
pass_type_1            0
pass_mat_1             0
pass_2                 0
pass_dis_2             0
pass_type_2            0
pass_mat_2             0
pass_3                 0
pass_dis_3             0
pass_type_3            0
pass_mat_3             0
pass_4                 0
pass_dis_4             0
pass_type_4            0
pass_mat_4             0
pass_5                 0
pass_dis_5             0
pass_type_5            0
pass_mat_5             0
pass_6                 0
pass_dis_6             0
pass_type_6            0
pass_mat_6             0


In [8]:
data.astype(np.float32)

,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,...,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start,path_loss,signal_after
0,2.4,0.0,0.0,0.0,-5.0,-5.0,-5.0,8.660254,180.0,0.0,...,0.000000,0.0,0.0,1.0,3.209568,2.0,1.0,246.0,46.939064,199.060928
1,5.0,0.0,0.0,0.0,-5.0,-5.0,-4.5,8.381527,180.0,270.0,...,0.000000,0.0,0.0,1.0,0.924371,3.0,1.0,242.0,52.182335,189.817673
2,2.4,0.0,0.0,0.0,-5.0,-5.0,-4.0,8.124039,180.0,270.0,...,4.412869,2.0,1.0,0.0,0.000000,0.0,0.0,244.0,43.800808,200.199188
3,5.0,0.0,0.0,0.0,-5.0,-5.0,-3.5,7.889867,90.0,90.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,242.0,51.690674,190.309326
4,2.4,0.0,0.0,0.0,-5.0,-5.0,-3.0,7.681146,0.0,0.0,...,1.955627,3.0,3.0,0.0,0.000000,0.0,0.0,253.0,51.233753,201.766251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,5.0,0.0,0.0,0.0,-3.0,-3.0,-2.5,4.924429,270.0,0.0,...,1.720892,4.0,2.0,1.0,2.762388,2.0,3.0,244.0,42.432251,201.567749
534,2.4,0.0,0.0,0.0,-3.0,-3.0,-2.0,4.690416,90.0,90.0,...,2.350213,3.0,3.0,1.0,4.466578,4.0,1.0,243.0,39.800808,203.199188
535,5.0,0.0,0.0,0.0,-3.0,-3.0,-1.5,4.500000,270.0,180.0,...,7.191378,3.0,1.0,1.0,3.130887,4.0,1.0,251.0,44.138367,206.861633
536,2.4,0.0,0.0,0.0,-3.0,-3.0,-1.0,4.358899,0.0,0.0,...,0.000000,0.0,0.0,1.0,2.554858,3.0,3.0,242.0,41.521515,200.478485


In [9]:
#again, ke is exit as we want to see how we can prevent this from occuring
##Splitting Independent and dependent variable in X and Y respectively
X = data.drop(['signal_after','path_loss'],axis=1)
Y = data[['path_loss']].values
Y2=data[['signal_after']].values
print(X.shape)
print(Y.shape)


(538, 53)
(538, 1)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.1)

In [11]:
X_train = X_train.astype(np.float64)

y_train = y_train.astype(np.float64)

In [12]:
# build the model!
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Dense(1000, input_shape=(X_train.shape[1],), activation='relu')) # (features,)
model.add(Dense(750, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(75, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='linear')) # output node
model.summary() # see what your model looks like

# compile the model
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

# early stopping callback
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=1000,
                   restore_best_weights = True)

# fit the model!
# attach it to a new variable called 'history' in case
# to look at the learning curves
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    callbacks=[es],
                    epochs=5000,
                    batch_size=50,
                    verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              54000     
                                                                 
 dense_1 (Dense)             (None, 750)               750750    
                                                                 
 dense_2 (Dense)             (None, 500)               375500    
                                                                 
 dense_3 (Dense)             (None, 250)               125250    
                                                                 
 dense_4 (Dense)             (None, 150)               37650     
                                                                 
 dense_5 (Dense)             (None, 75)                11325     
                                                                 
 dense_6 (Dense)             (None, 20)                1

In [13]:
X_train.dtypes

Frequency              float64
router_x               float64
router_y               float64
router_z               float64
cell_x                 float64
cell_y                 float64
cell_z                 float64
Distance               float64
orientation_x          float64
orientation_y          float64
orientation_z          float64
orientation_overall    float64
pass_0                 float64
pass_dis_0             float64
pass_type_0            float64
pass_mat_0             float64
pass_1                 float64
pass_dis_1             float64
pass_type_1            float64
pass_mat_1             float64
pass_2                 float64
pass_dis_2             float64
pass_type_2            float64
pass_mat_2             float64
pass_3                 float64
pass_dis_3             float64
pass_type_3            float64
pass_mat_3             float64
pass_4                 float64
pass_dis_4             float64
pass_type_4            float64
pass_mat_4             float64
pass_5  

In [14]:
# metrics
pred = model.predict(X_test)
pred

trainpreds = model.predict(X_train)

from sklearn.metrics import mean_absolute_error
pred_mae=mean_absolute_error(y_train, trainpreds)
print(mean_absolute_error(y_train, trainpreds)) # train
print(mean_absolute_error(y_test, pred)) # test


16/16 [==============================] - 0s 7ms/step
1.339974540652985
1.7006800940757019


In [15]:
data.head()

,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,...,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start,path_loss,signal_after
0,2.4,0,0,0,-5.0,-5.0,-5.0,8.660254,180,0,...,0.000000,0,0,1,3.209568,2,1,246,46.939065,199.060935
1,5.0,0,0,0,-5.0,-5.0,-4.5,8.381527,180,270,...,0.000000,0,0,1,0.924371,3,1,242,52.182334,189.817666
2,2.4,0,0,0,-5.0,-5.0,-4.0,8.124038,180,270,...,4.412869,2,1,0,0.000000,0,0,244,43.800806,200.199194
3,5.0,0,0,0,-5.0,-5.0,-3.5,7.889867,90,90,...,0.000000,0,0,0,0.000000,0,0,242,51.690673,190.309327
4,2.4,0,0,0,-5.0,-5.0,-3.0,7.681146,0,0,...,1.955627,3,3,0,0.000000,0,0,253,51.233753,201.766247


In [16]:
Predictors=['router_x',	'router_y',	'router_z',	'cell_x',	'cell_y',	'cell_z',	'Distance',	'orientation_x',	'orientation_y',	'orientation_z',
            'pass_0',	'pass_dis_0',	'pass_type_0',	'pass_mat_0',
            'pass_1',	'pass_dis_1',	'pass_type_1',	'pass_mat_1',
            'pass_2',	'pass_dis_2',	'pass_type_2',	'pass_mat_2',
            'pass_3',	'pass_dis_3',	'pass_type_3',	'pass_mat_3',
            'pass_4',	'pass_dis_4',	'pass_type_4',	'pass_mat_4',
            'pass_5',	'pass_dis_5',	'pass_type_5',	'pass_mat_5',
            'pass_6',	'pass_dis_6',	'pass_type_6',	'pass_mat_6',
            'pass_7',	'pass_dis_7',	'pass_type_7',	'pass_mat_7',
            'pass_8',	'pass_dis_8',	'pass_type_8',	'pass_mat_8',
            'signal_start']
TestingData=pd.DataFrame(data=X_test, columns=Predictors)
TestingData['Loss']=y_test
TestingData['Predicted_Loss']=pred
TestingData['Error']=abs(y_test-pred)/(y_test)*100
TestingData.head()

,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,orientation_z,...,pass_type_7,pass_mat_7,pass_8,pass_dis_8,pass_type_8,pass_mat_8,signal_start,Loss,Predicted_Loss,Error
418,0,0,0,-3.5,-2.5,-5.0,6.595453,180,90,270,...,0,0,1,3.964567,2,1,244,43.630841,43.419792,0.483716
490,0,0,0,-3.0,-5.0,-2.0,6.164414,180,0,90,...,1,2,1,3.180569,3,2,248,47.218392,47.025440,0.408637
518,0,0,0,-3.0,-3.5,-4.5,6.442049,180,0,180,...,1,3,0,0.000000,0,0,241,42.635243,45.152409,5.903955
357,0,0,0,-4.0,0.0,-2.5,4.716991,90,180,180,...,4,2,1,3.550415,2,1,248,39.662872,41.230522,3.952436
498,0,0,0,-3.0,-4.5,-3.5,6.442049,90,0,270,...,0,0,0,0.000000,0,0,250,47.742856,45.761398,4.150270


In [17]:
#Read in the data from excel (I converted form csv)
data2 = pd.read_excel('/content/drive/MyDrive/room1.xlsx')

In [18]:
data2.head()

,Router,Cell,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,...,pass_mat_7,pass_8,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start
0,By_couch,Couch,2.4,-0.5,0.0,1.0,-1.0,0.00,1.00,0.500000,...,0,0,0,0,0,0,0,0,0,240
1,By_couch,side_table_couch,2.4,-0.5,0.0,1.0,-1.0,0.50,1.00,0.707107,...,0,0,0,0,0,0,0,0,0,240
2,By_couch,table_sitting,2.4,-0.5,0.0,1.0,-0.5,-3.50,0.50,3.535534,...,0,0,0,0,0,0,0,0,0,240
3,By_couch,dining_table,2.4,-0.5,0.0,1.0,2.5,0.50,0.75,3.051639,...,0,0,0,0,0,0,0,0,0,240
4,By_couch,dining_table_2,2.4,-0.5,0.0,1.0,2.5,-3.75,0.75,4.808846,...,0,0,0,0,0,0,0,0,0,240


In [19]:
data2=data2.drop(['Router','Cell'],axis=1)
data2.head()

,Frequency,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,...,pass_mat_7,pass_8,pass_dis_8,pass_type_8,pass_mat_8,pass_9,pass_dis_9,pass_type_9,pass_mat_9,signal_start
0,2.4,-0.5,0.0,1.0,-1.0,0.00,1.00,0.500000,0,0,...,0,0,0,0,0,0,0,0,0,240
1,2.4,-0.5,0.0,1.0,-1.0,0.50,1.00,0.707107,0,0,...,0,0,0,0,0,0,0,0,0,240
2,2.4,-0.5,0.0,1.0,-0.5,-3.50,0.50,3.535534,0,0,...,0,0,0,0,0,0,0,0,0,240
3,2.4,-0.5,0.0,1.0,2.5,0.50,0.75,3.051639,0,0,...,0,0,0,0,0,0,0,0,0,240
4,2.4,-0.5,0.0,1.0,2.5,-3.75,0.75,4.808846,0,0,...,0,0,0,0,0,0,0,0,0,240


In [20]:
pred = model.predict(data2)
pred

5/5 [==============================] - 0s 6ms/step


array([[32.981876],
       [32.49103 ],
       [37.55125 ],
       [32.38691 ],
       [37.31847 ],
       [37.755047],
       [36.530426],
       [38.231403],
       [40.46113 ],
       [39.564625],
       [38.974346],
       [40.176163],
       [44.633583],
       [27.802021],
       [30.319525],
       [25.846304],
       [29.267336],
       [27.140057],
       [26.938765],
       [28.505888],
       [28.203493],
       [32.97699 ],
       [32.139343],
       [37.65159 ],
       [32.606144],
       [37.505363],
       [37.82317 ],
       [36.66634 ],
       [38.352985],
       [40.499226],
       [39.347687],
       [38.73676 ],
       [39.978092],
       [44.591084],
       [28.107052],
       [30.348448],
       [25.911026],
       [29.394112],
       [27.012314],
       [26.829475],
       [28.311447],
       [28.01304 ],
       [33.130512],
       [32.766163],
       [36.9264  ],
       [31.642065],
       [36.004753],
       [37.363514],
       [36.484123],
       [37.703568],


In [21]:
Predictors=['router_x',	'router_y',	'router_z',	'cell_x',	'cell_y',	'cell_z',	'Distance',	'orientation_x',	'orientation_y',	'orientation_z',
            'pass_0',	'pass_dis_0',	'pass_type_0',	'pass_mat_0',
            'pass_1',	'pass_dis_1',	'pass_type_1',	'pass_mat_1',
            'pass_2',	'pass_dis_2',	'pass_type_2',	'pass_mat_2',
            'pass_3',	'pass_dis_3',	'pass_type_3',	'pass_mat_3',
            'pass_4',	'pass_dis_4',	'pass_type_4',	'pass_mat_4',
            'pass_5',	'pass_dis_5',	'pass_type_5',	'pass_mat_5',
            'pass_6',	'pass_dis_6',	'pass_type_6',	'pass_mat_6',
            'pass_7',	'pass_dis_7',	'pass_type_7',	'pass_mat_7',
            'pass_8',	'pass_dis_8',	'pass_type_8',	'pass_mat_8',
            'signal_start']
TestingData=pd.DataFrame(data=data2, columns=Predictors)
TestingData['Predicted_Loss']=pred
TestingData.head()

,router_x,router_y,router_z,cell_x,cell_y,cell_z,Distance,orientation_x,orientation_y,orientation_z,...,pass_7,pass_dis_7,pass_type_7,pass_mat_7,pass_8,pass_dis_8,pass_type_8,pass_mat_8,signal_start,Predicted_Loss
0,-0.5,0.0,1.0,-1.0,0.00,1.00,0.500000,0,0,0,...,0,0,0,0,0,0,0,0,240,32.981876
1,-0.5,0.0,1.0,-1.0,0.50,1.00,0.707107,0,0,0,...,0,0,0,0,0,0,0,0,240,32.491032
2,-0.5,0.0,1.0,-0.5,-3.50,0.50,3.535534,0,0,0,...,0,0,0,0,0,0,0,0,240,37.551250
3,-0.5,0.0,1.0,2.5,0.50,0.75,3.051639,0,0,0,...,0,0,0,0,0,0,0,0,240,32.386909
4,-0.5,0.0,1.0,2.5,-3.75,0.75,4.808846,0,0,0,...,0,0,0,0,0,0,0,0,240,37.318470
